# Fetch raw data and validate

In [108]:
from pathlib import Path
import requests

from io import StringIO
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import os

def download_one_file_of_raw_data(year: int, month: int) -> Path:
    """
    """
    
    # Chrome driver settings
    option = webdriver.ChromeOptions()
    option.add_argument("start-maximized")
    prefs = {'download.default_directory' : os.getcwd()+'/data/raw'}
    option.add_experimental_option('prefs', prefs)

    # Load page
    URL = 'https://tso.nbpower.com/Public/en/system_information_archive.aspx'
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
    driver.get(URL)
    
    # Set menus
    month_sel = driver.find_element("xpath",'/html/body/form/div[5]/div[1]/table[1]/tbody/tr[2]/td[1]/select')
    year_sel = driver.find_element("xpath",'//*[@id="ctl00_cphMainContent_ddlYear"]')
    
    
    # Select year and month
    year_dict = {val.text: index for index,val in enumerate(year_sel.find_elements(By.TAG_NAME,"option"))}
    year_menu = year_sel.find_elements(By.TAG_NAME,"option")
    year_menu[year_dict[str(year)]].click()
    
    month_dict = {val.text: index for index,val in enumerate(month_sel.find_elements(By.TAG_NAME,"option"))}
    month_menu = month_sel.find_elements(By.TAG_NAME,"option")
    month_menu[month_dict[str(month)]].click()
    
    
    # View data
    view_data_button = driver.find_element("xpath",'//*[@id="ctl00_cphMainContent_lbGetData"]')
    view_data_button.click()
    
    # Parse data into data frame then write to parquet
    output_text=driver.find_element(By.TAG_NAME,"pre").text
    csvStringIO = StringIO(output_text)
    df = pd.read_csv(csvStringIO, sep=",", header=0)
    df.to_parquet(f'../data/raw/nb_power_{year}_{month:02d}.parquet')
    
    # if response.status_code == 200:
    #     path = f'../data/raw/rides_{year}-{month:02d}.parquet'
    #     open(path, "wb").write(response.content)
    #     return path
    # else:
    #     raise Exception(f'{URL} is not available')

In [109]:
download_one_file_of_raw_data(year=2023, month=1)

In [81]:
driver.find_element("xpath",'//*[@id="content"]/div/fieldset/h2').text

'404 - File or directory not found.'

In [2]:
URL = 'https://tso.nbpower.com/Public/en/system_information_archive.aspx'
response = requests.get(URL)

In [3]:
response.status_code

200

In [ ]:
https://tso.nbpower.com/Public/en/system_information_archive.aspx

In [ ]:
table xpath

//*[@id="aspnetForm"]/div[5]/div[1]/table[1]/tbody/tr[2]

In [5]:
import selenium
selenium.__version__

'4.17.2'

In [93]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import os

# driver = webdriver.Chrome(service=ChromiumService()
option = webdriver.ChromeOptions()
option.add_argument("start-maximized")

prefs = {'download.default_directory' : os.getcwd()+'/data/raw'}
option.add_experimental_option('prefs', prefs)


URL = 'https://tso.nbpower.com/Public/en/system_information_archive.aspx'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
driver.get(URL)

In [79]:
dir(driver)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_authenticator_id',
 '_check_if_window_handle_is_current',
 '_file_detector',
 '_get_cdp_details',
 '_is_remote',
 '_mobile',
 '_shadowroot_cls',
 '_switch_to',
 '_unwrap_value',
 '_web_element_cls',
 '_wrap_value',
 'add_cookie',
 'add_credential',
 'add_virtual_authenticator',
 'back',
 'bidi_connection',
 'capabilities',
 'caps',
 'close',
 'command_executor',
 'create_web_element',
 'current_url',
 'current_window_handle',
 'delete_all_cookies',
 'delete_cookie',
 'delete_downloadable_files',
 'delete_network_conditions',
 'download_file',
 'error_handl

In [94]:
month_sel = driver.find_element("xpath",'/html/body/form/div[5]/div[1]/table[1]/tbody/tr[2]/td[1]/select')
year_sel = driver.find_element("xpath",'//*[@id="ctl00_cphMainContent_ddlYear"]')
dload_button = driver.find_element("xpath",'//*[@id="ctl00_cphMainContent_lbGetData"]')

In [59]:
# year_sel = driver.find_element_by_id("XYEAR")
# all_years = year_sel.find_elements_by_tag_name("option")

In [100]:
{val.text: index for index,val in enumerate(year_sel.find_elements(By.TAG_NAME,"option"))}

{'2024': 0,
 '2023': 1,
 '2022': 2,
 '2021': 3,
 '2020': 4,
 '2019': 5,
 '2018': 6,
 '2017': 7}

In [101]:
{val.text: index for index,val in enumerate(month_sel.find_elements(By.TAG_NAME,"option"))}

{'1': 0,
 '2': 1,
 '3': 2,
 '4': 3,
 '5': 4,
 '6': 5,
 '7': 6,
 '8': 7,
 '9': 8,
 '10': 9,
 '11': 10,
 '12': 11}

In [98]:
year_dict = dict()
for index,val in enumerate(year_sel.find_elements(By.TAG_NAME,"option")):
    year_dict[val.text] = index
    
year_dict

{'2024': 0,
 '2023': 1,
 '2022': 2,
 '2021': 3,
 '2020': 4,
 '2019': 5,
 '2018': 6,
 '2017': 7}

In [61]:
[i.text for i in month_sel.find_elements(By.TAG_NAME,"option")]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

In [47]:
[i.text for i in year_sel.find_elements(By.TAG_NAME,"option")]

['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017']

In [87]:
year_sel.find_elements(By.TAG_NAME,"option")[4].click()

In [88]:
dload_button.click()

In [89]:
year=2020
month=5

output_text=driver.find_element(By.TAG_NAME,"pre").text

In [90]:


csvStringIO = StringIO(output_text)
df = pd.read_csv(csvStringIO, sep=",", header=0)
df.head()

,HOUR,NB_LOAD,NB_DEMAND,ISO_NE,NMISA,QUEBEC,NOVA_SCOTIA,PEI
0,2020-01-01 00:00,1711,1776,568,64,-943,128,26
1,2020-01-01 01:00,1649,1700,396,69,-666,155,17
2,2020-01-01 02:00,1625,1677,453,65,-838,156,30
3,2020-01-01 03:00,1607,1658,473,62,-829,155,5
4,2020-01-01 04:00,1590,1641,490,62,-832,154,-6


In [91]:
df.to_parquet(f'../data/raw/nb_power_{year}_{month:02d}.parquet')

In [70]:
with open(f'../data/raw/nb_power_{year}_{month:02d}.csv','wb') as file:
    for line in output_text:
        file.write(line)
        file.write('\n')

TypeError: a bytes-like object is required, not 'int'

In [66]:
driver.find_element(By.TAG_NAME,"pre").text

'HOUR,NB_LOAD,NB_DEMAND,ISO_NE,NMISA,QUEBEC,NOVA_SCOTIA,PEI\n2020-01-01 00:00,1711,1776,568,64,-943,128,26\n2020-01-01 01:00,1649,1700,396,69,-666,155,17\n2020-01-01 02:00,1625,1677,453,65,-838,156,30\n2020-01-01 03:00,1607,1658,473,62,-829,155,5\n2020-01-01 04:00,1590,1641,490,62,-832,154,-6\n2020-01-01 05:00,1603,1658,498,63,-869,154,-8\n2020-01-01 06:00,1638,1697,487,69,-900,155,-6\n2020-01-01 07:00,1692,1751,488,69,-859,156,8\n2020-01-01 08:00,1745,1810,505,73,-949,132,23\n2020-01-01 09:00,1804,1865,477,79,-949,134,33\n2020-01-01 10:00,1846,1910,472,82,-950,178,61\n2020-01-01 11:00,1856,1921,418,82,-968,178,74\n2020-01-01 12:00,1854,1918,361,88,-950,176,88\n2020-01-01 13:00,1844,1906,399,85,-950,132,89\n2020-01-01 14:00,1832,1893,404,82,-949,132,69\n2020-01-01 15:00,1834,1896,420,81,-949,132,64\n2020-01-01 16:00,1856,1923,414,75,-950,132,50\n2020-01-01 17:00,1924,1996,406,77,-950,131,81\n2020-01-01 18:00,1971,2048,451,85,-950,113,97\n2020-01-01 19:00,1939,2017,470,86,-958,112,95\n2

In [40]:
dir(year_sel)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

In [24]:
month_sel.id

'C9A54254D2EC820D8F635E650F35D15F_element_4'

In [30]:
dir(month_sel.find_element(By.TAG_NAME,"option"))

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

In [37]:
month_sel.find_element(By.TAG_NAME,"option").get_attribute("value")

'1'

In [19]:
dir(month_sel)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

In [22]:
month_sel.tag_name

'select'